# TTS Models Evaluation - Zero-Shot Voice Cloning

Este notebook analiza las métricas de rendimiento de los 3 modelos TTS implementados:
- **YourTTS**: Modelo multilingüe general
- **XTTS v2**: Modelo avanzado de alta calidad
- **VITS**: Modelo ligero y rápido

## Métricas Evaluadas

1. **RTF (Real-Time Factor)**: Tiempo de generación / Duración del audio
   - RTF < 1: Más rápido que tiempo real
   - RTF = 1: Tiempo real
   - RTF > 1: Más lento que tiempo real

2. **Generation Time**: Tiempo total de inferencia

3. **Audio Duration**: Duración del audio generado

## 1. Setup e Importación de Datos

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import numpy as np
from datetime import datetime

# Configuración de estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

In [ ]:
# Cargar todos los archivos JSON de resultados
results_dir = Path('../results')
json_files = sorted(results_dir.glob('metrics_*.json'))

print(f"Found {len(json_files)} result files:")
for f in json_files:
    print(f"  - {f.name}")

In [ ]:
# Función para cargar y procesar los datos
def load_metrics_from_json(json_path):
    """
    Carga métricas desde un archivo JSON.
    
    Returns:
        DataFrame con las métricas de cada modelo
    """
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Extraer información general
    timestamp = data.get('timestamp', 'unknown')
    text = data.get('text', '')
    reference = data.get('reference_audio', '')
    
    # Crear DataFrame con los resultados de cada modelo
    models_data = []
    for model in data.get('models', []):
        if model.get('success', False):
            models_data.append({
                'timestamp': timestamp,
                'text': text,
                'model': model['model'],
                'audio_duration': model['audio_duration'],
                'generation_time': model['generation_time'],
                'rtf': model['rtf'],
                'output_path': model['output_path'],
                'success': True
            })
        else:
            models_data.append({
                'timestamp': timestamp,
                'text': text,
                'model': model['model'],
                'error': model.get('error', 'Unknown error'),
                'success': False
            })
    
    return pd.DataFrame(models_data)

# Cargar todos los resultados
all_results = []
for json_file in json_files:
    df = load_metrics_from_json(json_file)
    all_results.append(df)

# Combinar todos los resultados
if all_results:
    metrics_df = pd.concat(all_results, ignore_index=True)
    print(f"\nLoaded {len(metrics_df)} model evaluations")
    display(metrics_df.head())
else:
    print("\n⚠️ No se encontraron archivos de métricas.")
    print("Ejecuta primero: make run-all TEXT='Your test text'")
    metrics_df = pd.DataFrame()

## 2. Estadísticas Generales

In [ ]:
if not metrics_df.empty:
    # Filtrar solo resultados exitosos
    successful_df = metrics_df[metrics_df['success'] == True].copy()
    
    if not successful_df.empty:
        print("=" * 60)
        print("ESTADÍSTICAS POR MODELO")
        print("=" * 60)
        
        # Agrupar por modelo y calcular estadísticas
        stats = successful_df.groupby('model').agg({
            'rtf': ['mean', 'std', 'min', 'max'],
            'generation_time': ['mean', 'std', 'min', 'max'],
            'audio_duration': ['mean', 'std']
        }).round(3)
        
        display(stats)
    else:
        print("⚠️ No hay resultados exitosos para analizar.")

## 3. Comparación de RTF (Real-Time Factor)

El RTF mide la eficiencia del modelo. Un RTF menor indica generación más rápida.

In [ ]:
if not metrics_df.empty and 'rtf' in successful_df.columns:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Crear gráfico de barras
    models = successful_df.groupby('model')['rtf'].mean().sort_values()
    colors = ['green' if rtf < 1 else 'orange' if rtf < 2 else 'red' for rtf in models]
    
    models.plot(kind='barh', ax=ax, color=colors)
    
    # Añadir línea de referencia en RTF=1
    ax.axvline(x=1, color='red', linestyle='--', linewidth=2, label='Real-time (RTF=1)')
    
    ax.set_xlabel('RTF (Real-Time Factor)', fontsize=12)
    ax.set_ylabel('Model', fontsize=12)
    ax.set_title('Comparación de Eficiencia: RTF por Modelo\n(Menor es mejor)', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Interpretación
    fastest = models.idxmin()
    print(f"\n🏆 Modelo más rápido: {fastest} (RTF: {models[fastest]:.2f}x)")
    
    if models[fastest] < 1:
        print(f"   → Genera audio {1/models[fastest]:.2f}x más rápido que tiempo real")
    else:
        print(f"   → Tarda {models[fastest]:.2f}x el tiempo del audio en generar")

## 4. Tiempo de Generación Absoluto

In [ ]:
if not metrics_df.empty and 'generation_time' in successful_df.columns:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Box plot para mostrar distribución
    successful_df.boxplot(column='generation_time', by='model', ax=ax)
    
    ax.set_xlabel('Model', fontsize=12)
    ax.set_ylabel('Generation Time (seconds)', fontsize=12)
    ax.set_title('Distribución de Tiempo de Generación por Modelo', fontsize=14, fontweight='bold')
    plt.suptitle('')  # Remove automatic title
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 5. Comparación Múltiple: RTF vs Generation Time

In [ ]:
if not metrics_df.empty:
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # RTF comparison
    rtf_data = successful_df.groupby('model')['rtf'].mean().sort_values()
    axes[0].barh(rtf_data.index, rtf_data.values, color='skyblue')
    axes[0].axvline(x=1, color='red', linestyle='--', linewidth=2, alpha=0.7)
    axes[0].set_xlabel('RTF', fontsize=12)
    axes[0].set_title('Real-Time Factor (RTF)', fontsize=13, fontweight='bold')
    axes[0].grid(True, alpha=0.3, axis='x')
    
    # Generation time comparison
    time_data = successful_df.groupby('model')['generation_time'].mean().sort_values()
    axes[1].barh(time_data.index, time_data.values, color='lightcoral')
    axes[1].set_xlabel('Seconds', fontsize=12)
    axes[1].set_title('Average Generation Time', fontsize=13, fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.show()

## 6. Análisis de Éxitos y Fallos

In [ ]:
if not metrics_df.empty:
    # Contar éxitos y fallos por modelo
    success_counts = metrics_df.groupby(['model', 'success']).size().unstack(fill_value=0)
    
    print("=" * 60)
    print("TASA DE ÉXITO POR MODELO")
    print("=" * 60)
    
    # Calcular porcentajes
    if True in success_counts.columns:
        success_counts['Success Rate (%)'] = (success_counts.get(True, 0) / 
                                               success_counts.sum(axis=1) * 100).round(2)
    
    display(success_counts)
    
    # Mostrar errores si existen
    failed_df = metrics_df[metrics_df['success'] == False]
    if not failed_df.empty:
        print("\n⚠️ ERRORES ENCONTRADOS:")
        print("=" * 60)
        for _, row in failed_df.iterrows():
            print(f"Model: {row['model']}")
            print(f"Error: {row.get('error', 'Unknown')}")
            print("-" * 60)

## 7. Resumen y Recomendaciones

In [ ]:
if not metrics_df.empty and not successful_df.empty:
    print("=" * 60)
    print("RESUMEN DE EVALUACIÓN")
    print("=" * 60)
    
    # Modelo más rápido (menor RTF)
    fastest_model = successful_df.groupby('model')['rtf'].mean().idxmin()
    fastest_rtf = successful_df.groupby('model')['rtf'].mean().min()
    
    # Modelo más consistente (menor std en RTF)
    most_consistent = successful_df.groupby('model')['rtf'].std().idxmin()
    consistency_std = successful_df.groupby('model')['rtf'].std().min()
    
    print(f"\n🏆 Modelo más rápido: {fastest_model}")
    print(f"   RTF promedio: {fastest_rtf:.2f}x")
    
    print(f"\n📊 Modelo más consistente: {most_consistent}")
    print(f"   Desviación estándar RTF: {consistency_std:.3f}")
    
    print("\n" + "=" * 60)
    print("RECOMENDACIONES")
    print("=" * 60)
    
    if fastest_rtf < 1:
        print(f"✓ {fastest_model} puede generar audio en tiempo real")
        print("  → Ideal para aplicaciones interactivas")
    
    print(f"\n✓ Para máxima velocidad: usar {fastest_model}")
    print(f"✓ Para resultados predecibles: usar {most_consistent}")
    
    # Análisis de calidad vs velocidad
    print("\n📝 NOTA: Este análisis solo considera métricas de rendimiento (velocidad).")
    print("   Para evaluar calidad de audio, considera:")
    print("   - Speaker Similarity (Resemblyzer, ECAPA-TDNN)")
    print("   - Audio Quality (PESQ, STOI)")
    print("   - Evaluación subjetiva (MOS)")

## 8. Exportar Resultados

In [ ]:
if not metrics_df.empty:
    # Exportar a CSV
    output_csv = results_dir / 'metrics_summary.csv'
    successful_df.to_csv(output_csv, index=False)
    print(f"✓ Resumen exportado a: {output_csv}")
    
    # Exportar estadísticas agregadas
    if not successful_df.empty:
        summary_stats = successful_df.groupby('model').agg({
            'rtf': ['mean', 'std', 'min', 'max'],
            'generation_time': ['mean', 'std', 'min', 'max'],
            'audio_duration': ['mean']
        }).round(3)
        
        output_stats = results_dir / 'metrics_statistics.csv'
        summary_stats.to_csv(output_stats)
        print(f"✓ Estadísticas exportadas a: {output_stats}")

---

## Notas

Este notebook analiza las métricas básicas de rendimiento. Para una evaluación completa, considera implementar:

1. **Speaker Similarity**: Usar Resemblyzer o SpeechBrain ECAPA-TDNN para medir similitud de voz
2. **Audio Quality**: Usar PESQ y STOI para evaluar calidad perceptual
3. **Subjective Evaluation**: MOS (Mean Opinion Score) con evaluadores humanos

Para más información, consulta el README del proyecto.